# Oxford Micromechanics Group EasyHyperspyEDX
## Written by R J Scales

This Notebook was written for a consistent SEM EDX output from various softwares that are implimented.
Benefits are the following:
* Ability to analyse the data on any PC
* Linear EDX colormaps
* ROI analysis
* and more...

For more information contact me on Microsoft Teams or email:

robert.scales@materials.ox.ac.uk

https://www.researchgate.net/profile/Robert-Scales-5

---

This code can be run in one go but it is wise to run it cell-by-cell.
If you unfamiliar with Jupyter Notebooks, it works on cells which are like MATLAB sections.
A quick guide can be found here:
https://realpython.com/jupyter-notebook-introduction/

To completely refresh the Python Kernel, go to Kernel to see the various options.
I recommend "restart and clear output"

### Section 1: Importing in Libaries
* Need to have HyperSpy & NumPy
* Anaconda has NumPy already
* https://hyperspy.org/hyperspy-doc/current/user_guide/install.html

In [1]:
# import hyperspy.io_plugins as pg
import hyperspy.api as hs
import hyperspy.io_plugins.bruker as bruker # Needed for *.bcf to be able to work
import numpy as np
import matplotlib.pyplot as plt
import os
# import PIL

%matplotlib qt

### Section 2: User Settings
The below are the variables which the user has to set or can optionally change.

**filename**: Variable to the fullfile location of the EDX file which you want to analyse.

So far the working filetypes are:
* *.bcf

In [41]:
filename = "C:/Documents/I9 Side Low Current x2k top left.bcf"
# All \ must be replaced with /

head, tail = os.path.split(filename)
tempTuple = os.path.splitext(tail)
Name = tempTuple[0]

mainpath = os.getcwd()
savepath = os.path.join(head, "HyperSpy", Name)

print(f'Operating on file called "{Name}"\nWorking directory: {mainpath}\nSaving directory: {savepath}')

try:
   os.makedirs(savepath)
except FileExistsError:
   # directory already exists
   pass

Operating on file called "I9 Side Low Current"
Working directory: C:\Users\mans3428\OneDrive - Nexus365\Documents\Python Scripts
Saving directory: //ajw-rs3618xs/AJWGroup_V8/Rob Scales SEM/EDX 2023-03-25\HyperSpy\I9 Side Low Current


**DS**: Stands for *downsample* which is spatial binning of data. Useful if you have not left the scan on for long enough.

In [42]:
DS = 1

**kV_max**: This is the highest energy which you want to cut-off the EDX spectrum to, e.g. 5 kV if you collected the EDX at 5 kV. 

In [43]:
kV_max = 5

**ColorMap**: Change the EDX color map to another valid *matplotlib* colormap. A recommended colormap is *viridis*.

In [44]:
ColorMap = 'viridis'

### Section 3: Loading Data and Plotting SE Image

In [45]:
s = hs.load(filename, select_type='spectrum_image', downsample=DS, cutoff_at_kV=kV_max)
s.plot()

### Section 4 (optional): Altering Elements in Loaded File to Analyse
If you run this section, then it will alter the element list based on the elements you have added.
I have mainly lifted from this page in the documentation and seems to be the best method I have found for this.

An example for where you want to analyse just Al, Ni, and Co with the automatic use of X-ray peaks to analyse for those elements based on the energy used.
```python
s.set_elements(['Al', 'Ni', 'Co'])
s.set_microscope_parameters(beam_energy=kV_max)
s.set_lines([])
```

The following are also useful for this:

http://hyperspy.org/hyperspy-doc/v1.3/api/hyperspy._signals.eds.html#hyperspy._signals.eds.EDS_mixin.set_elements

http://hyperspy.org/hyperspy-doc/v1.3/api/hyperspy._signals.eds.html#hyperspy._signals.eds.EDS_mixin.add_elements

In [46]:
# For CMSX-4
# https://cannonmuskegon.com/wp-content/uploads/2018/08/Improved-3rd-Generation-Single-Crystal-Superalloy-CMSX-4%C2%AE-Plus-SLS.pdf
# ['Cr', 'Co', 'W', 'Mo', 'Ta', 'Re', 'Al', 'Ti', 'Hf', 'Ni']

# s.set_elements(['Co', 'Mo','Al','Ni', 'O'])
# s.set_microscope_parameters(beam_energy=kV_max)
# s.set_lines([])

### Section 5: Region of Interest (ROI)
The below calculates using NumPy the size of the EDX map and then can use that to select the whole image as the ROI.

If you want **whole map** then set *roi* to the following:
```python
roi = hs.roi.RectangularROI(left=0, right=p[1], top=0, bottom=p[0])
```

For a smaller region of width *Width* and height *Height* in pixels, then change the line to the following and the ROI can be moved around in the window:
```python
roi = hs.roi.RectangularROI(left=0, right=Width, top=0, bottom=Height)
```

In [47]:
p = np.shape(s.data)
print(f"Map dimension = [{p[1]} {p[0]}]")

roi = hs.roi.RectangularROI(left=0, right=p[1], top=0, bottom=p[0])
im_roi = roi.interactive(s, color="red")

Map dimension = [336 252]


### Section 6: Analysis
This selects the data as whatever was within the ROI, which is usually a point of one pixel so this method is beneficial.

Useful url:
http://hyperspy.org/hyperspy-doc/current/user_guide/visualisation.html

In [48]:
s_roi = roi(im_roi)

#### §6.a Plot All Elements in ROI - Individual Colorbars
Plots all of the elements added so far with individual scaling.

In [49]:
eds_maps = s_roi.get_lines_intensity()
hs.plot.plot_images(eds_maps, axes_decor='off', scalebar='all', cmap=ColorMap, colorbar='default') # , tight_layout=True
plt.savefig(os.path.join(savepath, Name+' HyperSpy Maps Multi Colorbar.png'), transparent=True, dpi=300)
plt.savefig(os.path.join(savepath, Name+' HyperSpy Maps Multi Colorbar.svg'), transparent=True)

#### §6.b Plot All Elements in ROI - Single Colorbar
Plots all of the elements added so far with individual scaling.

In [50]:
eds_maps = s_roi.get_lines_intensity()
hs.plot.plot_images(eds_maps, axes_decor='off', scalebar='all', cmap=ColorMap, colorbar='single') # , tight_layout=True
plt.savefig(os.path.join(savepath, Name+' HyperSpy Maps Single Colorbar.png'), transparent=True, dpi=300)
plt.savefig(os.path.join(savepath, Name+' HyperSpy Maps Single Colorbar.svg'), transparent=True)

#### §6.c Plot ROI Spectrum
Plots the sum of the EDX spectrum from the points within the ROI.

In [51]:
s_roi_sum = s_roi.sum((0,1))
s_roi_sum.add_lines()
s_roi_sum.plot(integration_windows='auto')
plt.savefig(os.path.join(savepath, Name+' HyperSpy Spectrum.svg'), transparent=True)
plt.savefig(os.path.join(savepath, Name+' HyperSpy Spectrum.png'), transparent=True, dpi=300)

Traceback (most recent call last):
  File "C:\Users\mans3428\AppData\Local\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 304, in process
    func(*args, **kwargs)
  File "C:\Users\mans3428\AppData\Local\anaconda3\lib\site-packages\hyperspy\drawing\figure.py", line 68, in _on_blit_draw
    self._background = fig.canvas.copy_from_bbox(fig.bbox)
AttributeError: 'FigureCanvasSVG' object has no attribute 'copy_from_bbox'


#### §6.d Plot All Elements in ROI Overlay
If you want to plot certain elements then do the following:
```python
Elements2Show=['Ni_La', 'O_Ka']
```
**OR** to keep it the same as previously to have all the elements previously specificied then do:
```python
Elements2Show=None
```

In [52]:
print(f"The list of elments you can show are the following:\n------>{s_roi.metadata.Sample.xray_lines}")
Elements2Show = ['Ni_La', 'O_Ka', 'C_Ka']
Elements2ShowStr = ' '.join(Elements2Show)
print(f"Elements to be plotted are:\n------>{Elements2ShowStr}")
eds_maps2 = s_roi.get_lines_intensity(xray_lines=Elements2Show)
Colors = ['006BA4', 'FF800E', 'ABABAB', '595959', '5F9ED1', 'C85200', '898989', 'A2C8EC', 'FFBC79', 'CFCFCF']
hs.plot.plot_images(eds_maps2,scalebar='all', overlay=True, suptitle=False, axes_decor='off')
plt.savefig(os.path.join(savepath, Name+' HyperSpy EDX Overlay '+Elements2ShowStr+'.png'), transparent=True, dpi=300)
plt.savefig(os.path.join(savepath, Name+' HyperSpy EDX Overlay '+Elements2ShowStr+'.svg'), transparent=True)

#### §6.e Plot One Element
Plots the sum of the EDX spectrum from the points within the ROI.

If you want to plot a certain element then do the following:
```python
Element2Show=['O_Ka']
```

In [53]:
Element2Show=['O_Ka']
eds_maps3 = s_roi.get_lines_intensity(xray_lines=Element2Show)
hs.plot.plot_images(eds_maps3, cmap=ColorMap, axes_decor='off', tight_layout=True)
plt.savefig(os.path.join(savepath, Name+' HyperSpy EDX Overlay '+Element2Show[0]+'.png'), transparent=True, dpi=300)
plt.savefig(os.path.join(savepath, Name+' HyperSpy EDX Overlay '+Element2Show[0]+'.svg'), transparent=True)

## APPENDICES
```python
s.save('spectrum_eg')
```

The below converts it to numpy data:
```python
s_roi_np = s_roi.data
```

**Saving HyperSpy data**
http://hyperspy.org/hyperspy-doc/v1.3/user_guide/io.html?highlight=save#saving-data-to-files

**Loading HyperSpy Daya**
http://hyperspy.org/hyperspy-doc/v1.3/user_guide/io.html?highlight=save#loading-files-the-load-function

http://hyperspy.org/hyperspy-doc/v1.3/api/hyperspy._signals.eds.html#hyperspy._signals.eds.EDS_mixin.get_lines_intensity

For the saving of figures the following resources are useful:
https://stackoverflow.com/questions/32428193/saving-matplotlib-graphs-to-image-as-full-screen
https://stackoverflow.com/questions/32428193/saving-matplotlib-graphs-to-image-as-full-screen